# Initial Exploration

This project is based on the original [DeepPiCar](https://github.com/dctian/DeepPiCar) project which simply uses a Raspberry Pi with a camera on a simple radio controlled car. A vary similiar project [DeepPicar-v2](https://github.com/mbechtel2/DeepPicar-v2) done by members of University of Kansas. The DeepPiCar has data as well as the angles embedded in the images while DeepPicar-v2 has modulated the angles to $\lbrace -30, 0, +30\rbrace$ degrees on its outputs. Both are using the convolutional neural network architecture of the [DAVE-2](https://developer.nvidia.com/blog/deep-learning-self-driving-cars/) so for now training will only be done on the DeepPiCar data and comparison with their model.

## Camera Inputs

This section will look at the images being supplied by the DeepPiCar project as well as some image distortions achievable. First thing is to import the required libraries.

In [1]:
from cv2 import cv2              # an open computer vision module
import glob                      # Regix searching of files
import imageio                   # loading images in numpy array
import matplotlib.pyplot as plt  # Plotting
%matplotlib notebook
import numpy as np               # Array/Tensors
import os                        # finding paths
import pandas as pd              # Dataframe for viewing
import PIL                       # Python Imaging Library
import sys                       # manipulating the path variable
import tensorflow as tf          # Neural Networks

With these modules imported add in the location of the training images as well as the location of the DeepPiCar neural network.

In [2]:
sys.path.append(os.path.abspath('../data/external/'))
sys.path.append(os.path.abspath('../models/'))

With the added locations we can look load the data into a numpy data frame as well as a numpy array of angles.

In [3]:
# See number of images
image_list = glob.glob("../data/external/DeepPicar-data/*.png")  # All images

N = len(image_list)

image_vec = None  # Input
index_vec = np.empty((N,), dtype=np.int32) # Index
angle_vec = np.empty((N,), dtype=np.float64)  # Output Target

for i, f in enumerate(image_list):
    # Read in file
    image = imageio.imread(f)
    index = int(f.split('_')[-2])  # Frame index
    angle = float(f.split('_')[-1].split('.')[0])  # in degrees
    
    # Create image storage if necessary
    if image_vec is None:
        image_vec = np.empty((N,) + image.shape, dtype=np.float64)
    
    # Store file information
    image_vec[i] = image
    index_vec[i] = index
    angle_vec[i] = angle
    
    del image
    
# Some constants for resizing
target_img_height = 66
target_img_width = 200

# Resize the images
print("Original image vector dimensions: %s" % str(image_vec.shape))

image_vec = tf.image.resize(  # Maybe replace in network with crop
    images=tf.convert_to_tensor(image_vec),
    size=[target_img_height, target_img_width]
).numpy()

print("New image vector dimensions: %s" % str(image_vec.shape))

print("Minimum angle: %.0f degrees" % np.min(angle_vec))
print("Maximum angle: %.0f degrees" % np.max(angle_vec))

Original image vector dimensions: (219, 240, 320, 3)
New image vector dimensions: (219, 66, 200, 3)
Minimum angle: 44 degrees
Maximum angle: 103 degrees


## Model Building

With the images loaded and reshaped to the DAVE-2 image size we create a DAVE-2 neural network model for initial comparison work.

In [4]:
def create_dave(input_shape):
    # Create tensorflow object
    dave_2_model = tf.keras.Sequential()

    # First is a normalization layer in DAVE-2 but I don't see it in DeepPicar-v2 model-5conv_3fc
    # Attempting to normalize over color
    dave_2_model.add(tf.keras.layers.LayerNormalization(
        axis=[1, 2, 3], trainable=False,
        input_shape=input_shape  # Specify ahead of time for compilation
    ))

    """
    Brief comment:
    * Both DeepPicar and DeepPicar-v2 use a 240x320 frame initially but DeepPicar-v2 reshapes=66x200
    * DAVE-2 uses 66x200
    * Either I need to downsample both or I just ignore it since the DeepPicar's match.
    * Going with matching. <- scratch, will match dimensions
    """

    # Second portion is the sequential 5x5 convolutions of increasing filters at 2x2 strides
    for i in range(3):
        dave_2_model.add(tf.keras.layers.Conv2D(
            filters=(24 + i*12),
            kernel_size=(5,5),
            strides=(2,2),
            padding='valid',
            data_format='channels_last',
            activation=tf.nn.relu
        ))

    # Third there is another grouping of Conv2D layers but a 3x3 kernel with no stride
    for i in range(2):
        dave_2_model.add(tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=(3,3),
            padding='valid',
            data_format='channels_last',
            activation=tf.nn.relu
        ))

    # Fourth and final part is a flattening and then reduction to a single output
    dave_2_model.add(tf.keras.layers.Flatten())
    for n_neurons in [100, 50, 10, 1]:
        dave_2_model.add(tf.keras.layers.Dense(
            units=n_neurons,
            activation=(tf.nn.tanh if n_neurons == 1 else tf.nn.relu)
        ))

    # Compiling the model to see the initial structure
    dave_2_model.compile(
        optimizer='adam',
        loss=['mse'],
        metrics=['mse']
    )
    
    return dave_2_model

dave_2_model = create_dave(image_vec.shape[1:])
dave_2_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization (LayerNo (None, 66, 200, 3)        79200     
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

## Model Training on DeepPicar

With the model compiled and built initially, we will train on the DeepPicar Dataset and then see the accuraccy against the DeepPicar-v2 dataset

In [5]:
angle_center = 90.
angle_half_span = 30.

y_fit = (angle_vec - angle_center)/angle_half_span  # 30.  # 

# ensure -1 to 1 bounds
y_fit[y_fit < -1] = -1
y_fit[y_fit > 1] = 1

x_fit = image_vec

training_history = dave_2_model.fit(
    x=x_fit, y=y_fit,
    verbose=1,
    epochs=100,
    validation_split=0.2,
)

Train on 175 samples, validate on 44 samples
Epoch 1/100
175/175 [==============================] - 4s 23ms/sample - loss: 0.3383 - mse: 0.3383 - val_loss: 0.0329 - val_mse: 0.0329
Epoch 2/100
175/175 [==============================] - 0s 921us/sample - loss: 0.1452 - mse: 0.1452 - val_loss: 0.0278 - val_mse: 0.0278
Epoch 3/100
175/175 [==============================] - 0s 860us/sample - loss: 0.0989 - mse: 0.0989 - val_loss: 0.0305 - val_mse: 0.0305
Epoch 4/100
175/175 [==============================] - 0s 952us/sample - loss: 0.0768 - mse: 0.0768 - val_loss: 0.0491 - val_mse: 0.0491
Epoch 5/100
175/175 [==============================] - 0s 910us/sample - loss: 0.0647 - mse: 0.0647 - val_loss: 0.0408 - val_mse: 0.0408
Epoch 6/100
175/175 [==============================] - 0s 948us/sample - loss: 0.0632 - mse: 0.0632 - val_loss: 0.0424 - val_mse: 0.0424
Epoch 7/100
175/175 [==============================] - 0s 891us/sample - loss: 0.0479 - mse: 0.0479 - val_loss: 0.0291 - val_mse: 0.02

Epoch 60/100
175/175 [==============================] - 0s 857us/sample - loss: 7.4490e-04 - mse: 7.4490e-04 - val_loss: 0.0482 - val_mse: 0.0482
Epoch 61/100
175/175 [==============================] - 0s 952us/sample - loss: 6.3082e-04 - mse: 6.3082e-04 - val_loss: 0.0513 - val_mse: 0.0513
Epoch 62/100
175/175 [==============================] - 0s 952us/sample - loss: 6.1377e-04 - mse: 6.1377e-04 - val_loss: 0.0514 - val_mse: 0.0514
Epoch 63/100
175/175 [==============================] - 0s 857us/sample - loss: 8.2599e-04 - mse: 8.2599e-04 - val_loss: 0.0512 - val_mse: 0.0512
Epoch 64/100
175/175 [==============================] - 0s 958us/sample - loss: 6.7828e-04 - mse: 6.7828e-04 - val_loss: 0.0538 - val_mse: 0.0538
Epoch 65/100
175/175 [==============================] - 0s 941us/sample - loss: 6.1798e-04 - mse: 6.1798e-04 - val_loss: 0.0507 - val_mse: 0.0507
Epoch 66/100
175/175 [==============================] - 0s 863us/sample - loss: 5.8672e-04 - mse: 5.8672e-04 - val_loss: 0.0

Before moving on, let us address training speeds. Another student working on iterations of DeepPicar-vx has been training using `Tensorflow v1`. They train using a batch size of 128 images and at 24 steps per epoch and consistently get 2 to 3 seconds per step i.e. over 128 images. This means at fastest 15,625 microseconds per sample while the compiled version in this notebook is achieving 858 microseconds per sample so there is about a 2 to 3 times speedup using a fairly average laptop but compiled tensorflow as opposed to a GoogleColab but noncompiled version. After the training we do some plotting to see the training history results.

In [6]:
loss_vec = np.array(training_history.history['loss'])
val_loss_vec = np.array(training_history.history['val_loss'])

epoch_vec = np.arange(loss_vec.shape[0]) + 1

fig, ax = plt.subplots(1, 1)

ax.plot(epoch_vec, loss_vec, color='C0', label="Training Loss")
ax.plot(epoch_vec, val_loss_vec, color='C1', label="Validation Loss")

ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

ax.set_xlabel("Training Epoch")
ax.set_ylabel("Mean Square Error")

ax.legend()

plt.show()

<IPython.core.display.Javascript object>

The training looks fine, maybe optimal epoch number is 50 epochs. Let us check visually to make sure the results look reasonable.

In [7]:
# Constant
frame_num = 130

x_base = 100
y_base = 60
radius = 20

p_angle = (angle_half_span*dave_2_model.predict(image_vec)[frame_num] + angle_center)/180*np.pi
t_angle = angle_vec[frame_num]/180*np.pi

fig, ax = plt.subplots(1, 1)

ax.imshow(image_vec[frame_num].astype(int))
ax.plot(  # Arrow might be better
    np.array([x_base, x_base + radius*np.cos(np.pi - p_angle)]),
    np.array([y_base, y_base - radius*np.sin(p_angle)]),
    linewidth=5,
    label=("Predicted Angle (%.1f)" % (180./np.pi*p_angle))
)

ax.plot(  # Arrow might be better
    np.array([x_base, x_base + radius*np.cos(np.pi - t_angle)]),
    np.array([y_base, y_base - radius*np.sin(t_angle)]),
    linewidth=5,
    color='C3',
    label=("Target Angle (%.1f)" % (180./np.pi*t_angle))
)

ax.set_title("Frame %i" % frame_num)
ax.set_xlabel("Pixel Width")
ax.set_ylabel("Pixel Height")
ax.legend()

plt.show()

<IPython.core.display.Javascript object>

## Domain Augmentation from DeepPicar to DeepPicar-v2

With the inital DAVE-2 model trained and giving good results, we will try it on the more relavent dataset to see if the angles are fairly close.

In [8]:
# Some constants for analyzing videos from DeepPicar-v2
N_FRAMES = 1000  # All avi files in DeepPicar-v2 are 1,000 frames
ORIG_IMG_HEIGHT = 240
ORIG_IMG_WIDTH = 320


# Define some helper functions
def key_from_video_filename(vf : str):
    directory_seperated = vf.split('\\')  # running on Windows
    fileno = int(directory_seperated[-1].split('-')[-1].split('.')[0])
    key_filename = 'out-key-%i.csv' % fileno
    return '\\'.join(directory_seperated[:-1] + [key_filename,])


def image_vector_from_video(vf : str, image_vector : np.ndarray):
    "Trying inplace"
    # Preallocate output
    # image_vector = np.empty((N_FRAMES, ORIG_IMG_HEIGHT, ORIG_IMG_WIDTH, 3))
    
    # Get the video vile into a video capture
    video_capture = cv2.VideoCapture(vf)
    
    # Loop through frames
    for i in range(N_FRAMES):
        _, image = video_capture.read()
        
        image_vector[i,:,:,:] = image  # Store image
        
        del image
    
    # Some resource freeing
    video_capture.release()
    del video_capture
    
    # return image_vector


# Grab video file list
dpv2_video_files = glob.glob("../data/external/DeepPicar-v2-data/*.avi")
N_VIDEOS = len(dpv2_video_files)
    
    
# Declare variables to images and angles
# dpv2_images = np.empty((N_VIDEOS*N_FRAMES, target_img_height, target_img_width, 3))  # MemoryError if N_VIDEOS*N_FRAMES
dpv2_angles = np.empty((N_VIDEOS*N_FRAMES,))
predicted_angles = np.empty((N_VIDEOS*N_FRAMES,))
dpv2_times = np.empty((N_VIDEOS*N_FRAMES,))

# Iterate through the avi files
for i, vf in enumerate(dpv2_video_files):
    # Load in csv of correct outputs
    temp = np.loadtxt(
        fname=key_from_video_filename(vf),
        delimiter=',',
        skiprows=1  # ignore the column titles
    )
    
    # assign correct outputs to arrays
    dpv2_times[i*N_FRAMES:(i+1)*N_FRAMES] = temp[:,0]/1e6  # converted microseconds to seconds
    dpv2_angles[i*N_FRAMES:(i+1)*N_FRAMES] = temp[:,2]  # In radians and in {-30deg, 0deg, 30deg}
    
    del temp
    
    # Grab images from video to do predictions on
    image_vec = np.empty((N_FRAMES, ORIG_IMG_HEIGHT, ORIG_IMG_WIDTH, 3))
    image_vector_from_video(vf, image_vec)
    # reshaped_image_vec = np.empty((image_vec.shape[0],) + (target_img_height, target_img_width,3))
    
    # Resize images to match DAVE-2, seems to break if 1000 images
    reshape_batch = 100
    for j in range(N_FRAMES//reshape_batch):
        temp = tf.image.resize(
            images=tf.convert_to_tensor(image_vec[j*reshape_batch:(j+1)*reshape_batch,:,:,:]),
            size=[target_img_height, target_img_width]
        ).numpy()
        
        # reshaped_image_vec[j*reshape_batch:(j+1)*reshape_batch] = temp
        
        # Do predictions
        # pred = dave_2_model.predict(x=reshaped_image_vec, verbose=(1 if i == 0 else 0))
        pred = dave_2_model.predict(x=temp, verbose=1)
        
        # Save predictions
        predicted_angles[
            (i*N_FRAMES + j*reshape_batch):(i*N_FRAMES + (j+1)*reshape_batch)
        ] = pred[:,0]
        
        del temp, pred
    
    # Outer loop cleanup
    del image_vec
    

100/100 [==============================] - 0s 395us/sample


Having gone through all the DeepPicar-v2 and predicted the angle output from the neural network model trained on the DeepPicar dataset, it is now time to see the accuracy of the steering wheel angle.

In [9]:
# Rescale to {-1, 0, 1} for DeepPicar-v2
dpv2_angles = np.round(dpv2_angles*180./np.pi/30.)

In [10]:
# Also scale into {-1, 0, 1} with rounding
pred_control = np.round(predicted_angles)
pred_control[pred_control < -1] = -1
pred_control[pred_control > 1] = 1

In [11]:
cross_correlation_matrix = np.zeros((3,3))  # correlation matrix to see matches
for i in range(3):
    for j in range(3):
        cross_correlation_matrix[i, j] = np.count_nonzero(np.logical_and(
            np.equal(dpv2_angles, i-1),
            np.equal(pred_control, j-1),
        ))
        
"""
Quick note:
* column is predicted angle from -30 to 30 in left to right.
* row is actual from -30 to 30 from top to bottom.
"""
print(cross_correlation_matrix)

# Printing out the model statistics of discrete controller in the augmented domain
n_total = pred_control.shape[0]
n_correct = np.trace(cross_correlation_matrix)
n_incorrect = n_total - n_correct
print("Model accuracy: %2.2f%%'" % (100*n_correct/n_total))

[[1261.   55.    0.]
 [4160.  300.    0.]
 [4410.  814.    0.]]
Model accuracy: 14.19%'


So a couple of brief comments on the attempted domain augmentation is that the training dataset seems to have a large left turn bias while the augmented dataset has a larger right turn bias with less than 15% being a left turn example but around 50% being a right turn. The potential solution is to introduce some image augmentation into the DeepPicar to overcome left bias as well as some generic image manipulations like zoom or pan.

## Training with Image Augmentation

Here is the list of image augmentiation that will be implemented to increase the dataset size:

1. Flip over image width.

In [12]:
# Preallocate new fitting inputs and output arrays
n_fit = x_fit.shape[0]
n_fit_aug = 2*n_fit
x_fit_aug = np.empty((n_fit_aug,) + x_fit.shape[1:])
y_fit_aug = np.empty((n_fit_aug,))

# Fill in original images and angles
x_fit_aug[:n_fit] = x_fit
y_fit_aug[:n_fit] = y_fit

In [13]:
# Add in the image flip
x_fit_aug[n_fit:2*n_fit] = x_fit[:,:,::-1,:]
y_fit_aug[n_fit:2*n_fit] = -y_fit

fig, ax = plt.subplots(1, 2)

ax[0].imshow(x_fit_aug[frame_num].astype(int))
ax[1].imshow(x_fit_aug[frame_num + n_fit].astype(int))

ax[0].set_title("Original")
ax[1].set_title("Horizontal Flip")

plt.show()

<IPython.core.display.Javascript object>

In [14]:
# Retrain a new Dave-2
dave_2_model = create_dave(x_fit_aug.shape[1:])
training_history = dave_2_model.fit(
    x=x_fit_aug, y=y_fit_aug,
    verbose=1,
    epochs=100,
    validation_split=0.2,
)

Train on 350 samples, validate on 88 samples
Epoch 1/100
350/350 [==============================] - 1s 4ms/sample - loss: 0.3867 - mse: 0.3867 - val_loss: 1.0445 - val_mse: 1.0445
Epoch 2/100
350/350 [==============================] - 0s 946us/sample - loss: 0.1944 - mse: 0.1944 - val_loss: 0.4999 - val_mse: 0.4999
Epoch 3/100
350/350 [==============================] - 0s 944us/sample - loss: 0.0862 - mse: 0.0862 - val_loss: 0.2027 - val_mse: 0.2027
Epoch 4/100
350/350 [==============================] - 0s 956us/sample - loss: 0.0465 - mse: 0.0465 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 5/100
350/350 [==============================] - 0s 996us/sample - loss: 0.0264 - mse: 0.0264 - val_loss: 0.0948 - val_mse: 0.0948
Epoch 6/100
350/350 [==============================] - 0s 954us/sample - loss: 0.0221 - mse: 0.0221 - val_loss: 0.0822 - val_mse: 0.0822
Epoch 7/100
350/350 [==============================] - 0s 994us/sample - loss: 0.0175 - mse: 0.0175 - val_loss: 0.0760 - val_mse: 0.076

In [15]:
loss_vec = np.array(training_history.history['loss'])
val_loss_vec = np.array(training_history.history['val_loss'])

epoch_vec = np.arange(loss_vec.shape[0]) + 1

fig, ax = plt.subplots(1, 1)

ax.plot(epoch_vec, loss_vec, color='C0', label="Training Loss")
ax.plot(epoch_vec, val_loss_vec, color='C1', label="Validation Loss")

ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

ax.set_xlabel("Training Epoch")
ax.set_ylabel("Mean Square Error")

ax.legend()

plt.show()

<IPython.core.display.Javascript object>

In [16]:
# Iterate through the DeepPicar-v2 to redo prediction angles    
for i, vf in enumerate(dpv2_video_files):   
    # Grab images from video to do predictions on
    image_vec = np.empty((N_FRAMES, ORIG_IMG_HEIGHT, ORIG_IMG_WIDTH, 3))
    image_vector_from_video(vf, image_vec)
    
    # Resize images to match DAVE-2, seems to break if 1000 images
    reshape_batch = 100
    for j in range(N_FRAMES//reshape_batch):
        temp = tf.image.resize(
            images=tf.convert_to_tensor(image_vec[j*reshape_batch:(j+1)*reshape_batch,:,:,:]),
            size=[target_img_height, target_img_width]
        ).numpy()
        
        # reshaped_image_vec[j*reshape_batch:(j+1)*reshape_batch] = temp
        
        # Do predictions
        pred = dave_2_model.predict(x=temp, verbose=1)
        
        # Save predictions
        predicted_angles[
            (i*N_FRAMES + j*reshape_batch):(i*N_FRAMES + (j+1)*reshape_batch)
        ] = pred[:,0]
        
        del temp, pred
    
    # Outer loop cleanup
    del image_vec
    

100/100 [==============================] - 0s 459us/sample


In [17]:
pred_control = np.round(predicted_angles)
pred_control[pred_control < -1] = -1
pred_control[pred_control > 1] = 1

cross_correlation_matrix = np.zeros((3,3))  # correlation matrix to see matches
for i in range(3):
    for j in range(3):
        cross_correlation_matrix[i, j] = np.count_nonzero(np.logical_and(
            np.equal(dpv2_angles, i-1),
            np.equal(pred_control, j-1),
        ))
        
"""
Quick note:
* column is predicted angle from -30 to 30 in left to right.
* row is actual from -30 to 30 from top to bottom.
"""
print(cross_correlation_matrix)

# Printing out the model statistics of discrete controller in the augmented domain
n_total = pred_control.shape[0]
n_correct = np.trace(cross_correlation_matrix)
n_incorrect = n_total - n_correct
print("Model accuracy: %2.2f%%'" % (100*n_correct/n_total))

[[ 141. 1147.   28.]
 [ 820. 3334.  306.]
 [ 850. 3591.  783.]]
Model accuracy: 38.71%'


The accuracy of the model has dramatically improved although still undesirable but still need to verify the quality. It is unsure whether the wheel angles are an accurate representation.

In [19]:
# Video constants
video_save_location = "../reports/video-quality-checks/image-to-point/"
AVI_WIDTH = 360
AVI_HEIGHT = 240


# Plotting constants
xbase = 180
ybase = 220
r = 120
r_text = r + 10
font = cv2.FONT_HERSHEY_SIMPLEX

# Iterate through the DeepPicar-v2 to make a video for quality analysis
for i, vf in enumerate(dpv2_video_files): 
    
    # Save file name and reader ready
    vf_name = vf.split('\\')[-1]
    writer = None
    
    # Set up reading video
    cap = cv2.VideoCapture(vf)
    current_frame = 0
    
    # Read all the video
    while cap.isOpened():
        if current_frame < N_FRAMES:
            # Read in a frame from the video
            ret, frame = cap.read()

            # A quit option
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            # Initialize writer if necessary
            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*'DIVX')
                output_path = video_save_location + vf_name
                writer = cv2.VideoWriter(
                    output_path,
                    fourcc,  # color code
                    30,  # FPS
                    (frame.shape[1], frame.shape[0]),  # Image Shape
                    True  # in color
                )

            # Resize the image
            # frame = cv2.resize(src=frame, dsize=(target_img_height, target_img_width))
                
            # Calculations for prediction on frame
            theta = predicted_angles[i*N_FRAMES + current_frame]
            
            dx = int(r*np.sin(theta))
            dy = int(-r*np.cos(theta))
            dx_t = int(r_text*np.sin(theta))
            dy_t = int(-r_text*np.cos(theta))
            
            # Plot prediction on the frame
            frame = cv2.line(
                frame,
                (xbase, ybase),
                (xbase + dx, ybase + dy),
                (255,)*3,
                5
            )
            frame = cv2.putText(
                frame,
                'P',
                (xbase + dx_t, ybase + dy_t),
                font,
                2,
                (255,)*3,
                cv2.LINE_AA
            )
            
            # Calculations for actual on frame
            theta = dpv2_angles[i*N_FRAMES + current_frame]*np.pi/6.
            
            dx = int(r*np.sin(theta))
            dy = int(-r*np.cos(theta))
            dx_t = int(r_text*np.sin(theta))
            dy_t = int(-r_text*np.cos(theta))
            
            # Plot prediction on the frame
            frame = cv2.line(
                frame,
                (xbase, ybase),
                (xbase + dx, ybase + dy),
                (255,) + (0,)*2,
                5
            )
            frame = cv2.putText(
                frame,
                'A',
                (xbase + dx_t, ybase + dy_t),
                font,
                2,
                (255,) + (0,)*2,
                cv2.LINE_AA
            )

            # Write out and increment
            writer.write(frame)
            current_frame += 1  # Increment a counter

            # Delete frame
            del frame
            
        else:
            writer.release()
            cap.release()
            cv2.destroyAllWindows()
            break

## Conclusions

After review the quality videos, it fairly clear that the DeepPicar dataset will only deal with continuous left turns. DeepPicar-v2 will be a better training set even if it is discrete angles. 